In [ ]:
from utils import *
from plotly.subplots import make_subplots
from tqdm.auto import tqdm
import json
from loaders import *

In [ ]:
res_p_df, scalars_p = final_prelim_loader()
res_ad_df, scalars_ad = adaptive_ratio_wm_v1_0_loader()
res_ppo_df, scalars_ppo = ppo_sweep_loader()
ref_df = reference_loader()

In [ ]:
envs = res_p_df["env"].unique()

fig = make_subplots(
    cols=len(envs),
    column_titles=[*envs],
    shared_xaxes=True,
)

colors = make_color_iter()
colors = [next(colors) for _ in range(2)]

axis = 1
for col, env in enumerate(envs, 1):
    df1 = res_p_df[res_p_df["env"] == env].copy()
    tags = []
    for _, row in df1.iterrows():
        tag = f"{row['rl_ratio']}/{row['num_epochs']}"
        tags.append(tag)
    df1["tag"] = tags
    df1 = df1.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df1["tag"],
            y=df1["score"],
            sizemode="sd",
            boxmean=True,
            showlegend=False,
        ),
        row=1,
        col=col,
    )

    fig.update_layout(**{f"xaxis{axis}": dict(title="Configuration")}),
    axis += 1

fig.update_layout(
    boxmode="group",
    width=1000,
    height=400,
    yaxis_title="Score",
)
fig.write_image("../tex/assets/final_prelim.pdf")
fig

In [ ]:
envs = res_p_df["env"].unique()

fig = make_subplots(
    rows=len(envs),
    row_titles=[*envs],
)

colors = make_color_iter()
colors = [next(colors) for _ in range(4)]

axis = 1
for row_num, env in enumerate(envs, 1):
    df1 = res_p_df[res_p_df["env"] == env].copy()
    tags = []
    for _, row in df1.iterrows():
        tag = f"{row['rl_ratio']}/{row['num_epochs']}"
        tags.append(tag)
    df1["tag"] = tags
    df1 = df1.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df1["tag"],
            y=df1["score"],
            sizemode="sd",
            boxmean=True,
            legendgroup="0",
            name="Combined",
            showlegend=(row_num == 1),
            offsetgroup="0",
            marker=dict(color=colors[0]),
        ),
        row=row_num,
        col=1,
    )

    df2 = res_ad_df[res_ad_df["env"] == env].copy()
    tags = []
    for _, row in df2.iterrows():
        tag = f"K = {row['rl_ratio']}"
        tags.append(tag)
    df2["tag"] = tags
    df2 = df2.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df2["tag"],
            y=df2["score"],
            sizemode="sd",
            boxmean=True,
            legendgroup="1",
            name="Adaptive Ratio",
            showlegend=(row_num == 1),
            offsetgroup="0",
            marker=dict(color=colors[1]),
        ),
        row=row_num,
        col=1,
    )

    df3 = res_ppo_df[res_ppo_df["env"] == env].copy()
    tags = []
    for _, row in df3.iterrows():
        tag = f"{row['rl_ratio']}/{row['num_epochs']}/{row['num_mb']}"
        tags.append(tag)
    df3["tag"] = tags
    df3 = df3.sort_values(by="tag")
    fig.add_trace(
        go.Box(
            x=df3["tag"],
            y=df3["score"],
            sizemode="sd",
            boxmean=True,
            legendgroup="2",
            name="PPO",
            showlegend=(row_num == 1),
            offsetgroup="0",
            marker=dict(color=colors[2]),
        ),
        row=row_num,
        col=1,
    )

    fig.update_layout(**{f"yaxis{axis}": dict(title="Score")}),
    fig.update_layout(**{f"xaxis{axis}": dict(title="Configuration")}),
    axis += 1

fig.update_layout(
    boxmode="group",
    width=1000,
    height=800,
)
fig.write_image("../tex/assets/final_prelim_comp.pdf")
fig

In [ ]:
df1 = res_p_df.copy()
df1["group"] = "Combined"
tags = []
for _, row in df1.iterrows():
    tag = f"{row['rl_ratio']}/{row['num_epochs']}"
    tags.append(tag)
df1["tag"] = tags
df1 = df1.sort_values(by="tag")

df2 = res_ad_df.copy()
df2["group"] = "Adaptive"
tags = []
for _, row in df2.iterrows():
    tag = f"K = {row['rl_ratio']}"
    tags.append(tag)
df2["tag"] = tags
df2 = df2.sort_values(by="tag")

df3 = res_ppo_df.copy()
df3["group"] = "PPO"
tags = []
for _, row in df3.iterrows():
    tag = f"{row['rl_ratio']}/{row['num_epochs']}/{row['num_mb']}"
    tags.append(tag)
df3["tag"] = tags
df3 = df3.sort_values(by="tag")

df = pd.concat([df1, df2, df3])

g = df.groupby(["env", "group", "tag"])
df = g["score"].mean()
df = df.reset_index()

df = df.merge(right=ref_df, left_on="env", right_on="task")
df["norm"] = (df["score"] - df["random"]) / (df["human_gamer"] - df["random"])

df = df.rename(
    columns={
        "env": "Environment",
        "group": "Ablation",
        "tag": "Preset",
        "score": "Score",
    }
)

tab = df.pivot(index=["Ablation", "Preset"], columns="Environment", values="Score")
tab = tab.astype(int)

tab["Human Mean"] = df.groupby(["Ablation", "Preset"])["norm"].mean()
tab["Human Median"] = df.groupby(["Ablation", "Preset"])["norm"].median()

tab = tab.astype(object)
tab["Human Mean"] = tab["Human Mean"].apply(lambda x: f"{x:.2f}")
tab["Human Median"] = tab["Human Median"].apply(lambda x: f"{x:.2f}")

index = [
    *(("Environment", x) for x in df["Environment"].unique()),
    *(("Human Norm", x) for x in ("Human Mean", "Human Median")),
]
tab.columns = pd.MultiIndex.from_tuples(index)
tab = tab.rename(columns={"Human Mean": "Mean", "Human Median": "Median"})

for col in tab.columns:
    idx = tab.index[tab[col].argmax()]
    tab.loc[idx, col] = f"{{\\bf {tab.loc[idx, col]}}}"


with open("../tex/assets/final_prelim_tab.tex", "w") as f:
    tab.to_latex(f)

tab